# House Prices - Advanced regression techniques

In [1]:
import numpy as np
import pandas as 
import xgboost as xgb

In [55]:
np.set_printoptions(threshold=100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [44]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [50]:
print('train', train.shape)
print('test', test.shape)

train (1460, 81)
test (1459, 80)


In [45]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [51]:
y = 'SalesPrice'

In [46]:
from sklearn.preprocessing import LabelEncoder

In [47]:
catcols = train.columns[train.dtypes == object]

In [49]:
for col in catcols:
    if (~test[col].isin(train[col].unique())).sum() > 0:
        print(col)
        print(test.loc[~test[col].isin(train[col].unique()), col])

MSZoning
455     NaN
756     NaN
790     NaN
1444    NaN
Name: MSZoning, dtype: object
Utilities
455    NaN
485    NaN
Name: Utilities, dtype: object
Exterior1st
691    NaN
Name: Exterior1st, dtype: object
Exterior2nd
691    NaN
Name: Exterior2nd, dtype: object
KitchenQual
95    NaN
Name: KitchenQual, dtype: object
Functional
756     NaN
1013    NaN
Name: Functional, dtype: object
SaleType
1029    NaN
Name: SaleType, dtype: object


In [53]:
for col in catcols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col], test[col].fillna('NA')], axis=0))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col].fillna('NA'))

In [56]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,4,65.0,8450,1,3,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,1,13,14,1,196.0,2,4,2,2,4,4,2,706,6,0,150,856,1,0,1,4,856,854,0,1710,1,0,2,1,3,1,2,8,7,0,6,1,2003.0,2,2,548,5,5,2,0,61,0,0,0,0,4,5,5,0,2,2008,9,4,208500
1,2,20,4,80.0,9600,1,3,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,1,8,8,3,0.0,3,4,1,2,4,1,0,978,6,0,284,1262,1,0,1,4,1262,0,0,1262,0,1,2,0,3,1,4,6,7,1,5,1,1976.0,2,2,460,5,5,2,298,0,0,0,0,0,4,5,5,0,5,2007,9,4,181500
2,3,60,4,68.0,11250,1,3,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,1,13,14,1,162.0,2,4,2,2,4,2,2,486,6,0,434,920,1,0,1,4,920,866,0,1786,1,0,2,1,3,1,2,6,7,1,5,1,2001.0,2,2,608,5,5,2,0,42,0,0,0,0,4,5,5,0,9,2008,9,4,223500
3,4,70,4,60.0,9550,1,3,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,1,14,16,3,0.0,3,4,0,4,1,4,0,216,6,0,540,756,1,2,1,4,961,756,0,1717,1,0,1,0,3,1,2,7,7,1,2,5,1998.0,3,3,642,5,5,2,0,35,272,0,0,0,4,5,5,0,2,2006,9,0,140000
4,5,60,4,84.0,14260,1,3,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,1,13,14,1,350.0,2,4,2,2,4,0,2,655,6,0,490,1145,1,0,1,4,1145,1053,0,2198,1,0,2,1,4,1,2,9,7,1,5,1,2000.0,2,3,836,5,5,2,192,84,0,0,0,0,4,5,5,0,12,2008,9,4,250000
